# Imports

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from imblearn.over_sampling import SMOTE
import pandas as pd
import os

## Constants

In [2]:
parent_dir = r"../data/54ft/"
train_df_name = r"54ft_train.csv"

#feature # binding
F=\
    {1:'stay_duration', #f1
     2:'mfcc0',3:'mfcc1',4:'mfcc2',5:'mfcc3',6:'mfcc4', #f2,f3,f4,f5,f6
     7:'wifi_count', 8:'edge_wifi_count', #f7,f8
     9:'RSI', #f9
     10:'human_made', 11:'natural_land',12:'road_exist_percent',13:'highly_populated_poi_exist'#f10,f11,f12,f13
    }

selected_feat=\
    {
        'Is_Bus_stop':[F[e] for e in [9,10,11,12,13]],
        'Is_Turn':[F[e] for e in [9,10,11,12,13]],
        'Is_Signal':[F[e] for e in [9,10,11,12,13]],
        'Is_Congestion':[F[e] for e in [9,10,11,12,13]],
        'Is_Adhoc':[F[e] for e in [9,10,11,12,13]]
    }


#file Names
zone_names=list(map(lambda e:f"{e}_test_split.csv",["station","dvc_more","54ft_road","junction_mall","prantika_bus_stand"]))

_7day_test=["54ft_test.csv"]

In [3]:
#Modeling function
def get_metrics_from_the_model(train,labels_train,test,labels_test,poi,zone):
    rf=RandomForestClassifier(n_estimators=100,max_depth=8,random_state=42)
    rf.fit(train, labels_train)

    pred_train= rf.predict(train)
    pred_test= rf.predict(test)
    
    performance={'poi':poi,'zone':zone,
                 'tr_acc':accuracy_score(labels_train,pred_train),
                 'tr_precision':precision_score(labels_train,pred_train,average='weighted'),
                 'tr_recall':recall_score(labels_train,pred_train,average='weighted'),
                 'tr_f1-score':f1_score(labels_train,pred_train,average='weighted'),

                 'te_acc':accuracy_score(labels_test,pred_test),
                 'te_precision':precision_score(labels_test, pred_test,average='weighted'),
                 'te_recall':recall_score(labels_test, pred_test,average='weighted'),
                 'te_f1-score':f1_score(labels_test, pred_test,average='weighted')}
    return performance,pred_test
    

## Modeling

In [4]:
perf=[] #performance list


for poi_column in ['Is_Bus_stop','Is_Turn','Is_Signal','Is_Congestion','Is_Adhoc']:
    feature_names=selected_feat[poi_column] #feature selection

    for test_df_name in (zone_names+_7day_test):#iterator name

        zoneName=test_df_name.split('_test_split.csv')[0].split('.')[0]
        #output_result_filename = f"{poi_column}_test_result_{zoneName}_zone.csv"

        #train_data_processing
        train_csv_df = pd.read_csv(os.path.join(parent_dir, train_df_name))
        train_df = train_csv_df[feature_names+[poi_column]]

        #test data processing
        test_csv_df = pd.read_csv(os.path.join(parent_dir, test_df_name))
        test_df = test_csv_df[feature_names+[poi_column]]


        #SMOTE on training data & get features ,labels
        X = train_df[feature_names].copy()
        y = train_df[poi_column].copy()

        smote = SMOTE()
        X_resampled, y_resampled = smote.fit_sample(X, y)
        train_data = X_resampled.copy()
        train_labels = y_resampled.copy()

        #Test set get features & labels
        test_data = test_df[feature_names].values
        test_labels = test_df[poi_column].values

        #Training with Random Forest
        performance,pred_test=get_metrics_from_the_model(train_data,train_labels,test_data,test_labels,poi_column,zoneName)

        #adding to performance list
        perf.append(performance)

        #creating prediction file
        # test_csv_df[f'Prediction {poi_column}']=pred_test
        # result_df=test_csv_df[['start_date','start_time','end_time',f'Prediction {poi_column}']].copy()
        # result_df.columns=['instance_date', 'instance_start_time', 'instance_end_time',f'Prediction {poi_column}']
        # result_df.to_csv(os.path.join(parent_dir, output_result_filename), index=False)

        print(f'Completed for POI:{poi_column} and Zone:{zoneName}') #get results of length:{result_df.shape[0]}')

Completed for POI:Is_Bus_stop and Zone:station
Completed for POI:Is_Bus_stop and Zone:dvc_more
Completed for POI:Is_Bus_stop and Zone:54ft_road
Completed for POI:Is_Bus_stop and Zone:junction_mall
Completed for POI:Is_Bus_stop and Zone:prantika_bus_stand
Completed for POI:Is_Bus_stop and Zone:54ft_test
Completed for POI:Is_Turn and Zone:station
Completed for POI:Is_Turn and Zone:dvc_more
Completed for POI:Is_Turn and Zone:54ft_road
Completed for POI:Is_Turn and Zone:junction_mall
Completed for POI:Is_Turn and Zone:prantika_bus_stand
Completed for POI:Is_Turn and Zone:54ft_test
Completed for POI:Is_Signal and Zone:station
Completed for POI:Is_Signal and Zone:dvc_more
Completed for POI:Is_Signal and Zone:54ft_road
Completed for POI:Is_Signal and Zone:junction_mall
Completed for POI:Is_Signal and Zone:prantika_bus_stand
Completed for POI:Is_Signal and Zone:54ft_test
Completed for POI:Is_Congestion and Zone:station
Completed for POI:Is_Congestion and Zone:dvc_more
Completed for POI:Is_Cong

# Saving Performance for all Modelings

In [5]:
df=pd.DataFrame(perf)
df.to_csv(os.path.join(parent_dir, 'modeling_performance_spatial.csv'),index=False)
df

,poi,zone,tr_acc,tr_precision,tr_recall,tr_f1-score,te_acc,te_precision,te_recall,te_f1-score
0,Is_Bus_stop,station,0.935333,0.936638,0.935333,0.935285,0.740000,0.791133,0.740000,0.740936
1,Is_Bus_stop,dvc_more,0.932000,0.933631,0.932000,0.931936,0.896104,0.896104,0.896104,0.896104
2,Is_Bus_stop,54ft_road,0.938667,0.939680,0.938667,0.938631,0.921788,0.928155,0.921788,0.920451
3,Is_Bus_stop,junction_mall,0.933667,0.934586,0.933667,0.933632,0.960526,0.962533,0.960526,0.959441
4,Is_Bus_stop,prantika_bus_stand,0.930000,0.931695,0.930000,0.929931,0.942529,0.931440,0.942529,0.936952
5,Is_Bus_stop,54ft_test,0.931000,0.932736,0.931000,0.930931,0.873622,0.884774,0.873622,0.875023
6,Is_Turn,station,0.913472,0.915033,0.913472,0.913391,0.980000,0.980435,0.980000,0.978999
7,Is_Turn,dvc_more,0.910460,0.912784,0.910460,0.910334,0.740260,0.728390,0.740260,0.719184
8,Is_Turn,54ft_road,0.907174,0.908853,0.907174,0.907079,0.949721,0.961981,0.949721,0.953255
9,Is_Turn,junction_mall,0.906079,0.908053,0.906079,0.905965,0.934211,0.933103,0.934211,0.933421


In [6]:
#NICE

# On only Testing Dataset

In [7]:
df_pref_testset=df[df.zone=='54ft_test'].copy()
df_pref_testset.to_csv(os.path.join(parent_dir, 'modeling_performance_on_testset_spatial.csv'),index=False)
df_pref_testset

,poi,zone,tr_acc,tr_precision,tr_recall,tr_f1-score,te_acc,te_precision,te_recall,te_f1-score
5,Is_Bus_stop,54ft_test,0.931000,0.932736,0.931000,0.930931,0.873622,0.884774,0.873622,0.875023
11,Is_Turn,54ft_test,0.906079,0.908311,0.906079,0.905950,0.864071,0.888625,0.864071,0.869257
17,Is_Signal,54ft_test,0.935925,0.942052,0.935925,0.935703,0.861866,0.948779,0.861866,0.891200
23,Is_Congestion,54ft_test,0.858227,0.864320,0.858227,0.857632,0.603233,0.959858,0.603233,0.731588
29,Is_Adhoc,54ft_test,0.866193,0.866267,0.866193,0.866186,0.739162,0.740128,0.739162,0.734675


In [8]:
print('Mean f1:',df_pref_testset['te_f1-score'].mean(),'Std f1:',df_pref_testset['te_f1-score'].std())

Mean f1: 0.8203486319911791 Std f1: 0.08003064801609631


In [9]:
#DONE